# 第３回

##  xpathを使ったウェブスクレイピング

In [28]:
from lxml import html

with open("./c3.html", 'r') as f:
    fstr = f.read()

# lxmlでHTMLを解析
tree = html.fromstring(fstr)

# XPathでニュースタイトルを取得（GoogleChrome上の）

# どちらでも取れるのを確認
#titles = tree.xpath('/html/body/div/text()')
titles = tree.xpath('//div/text()')

# 結果を出力
for title in titles:
    print(title)


こんにちは世界



In [19]:
#response.content

## クローリング

### APIの例（郵便番号検索API）

In [83]:
import requests

In [84]:
res = requests.get("https://zipcloud.ibsnet.co.jp/api/search", params={'zipcode': '8701124'})

In [86]:
resDic = res.json()

In [87]:
resDic

{'message': None,
 'results': [{'address1': '大分県',
   'address2': '大分市',
   'address3': '旦野原',
   'kana1': 'ｵｵｲﾀｹﾝ',
   'kana2': 'ｵｵｲﾀｼ',
   'kana3': 'ﾀﾞﾝﾉﾊﾙ',
   'prefcode': '44',
   'zipcode': '8701124'}],
 'status': 200}

In [88]:
adDic1 = resDic["results"][0]

In [93]:
print(f"{adDic1['address1']}{adDic1['address2']}{adDic1['address3']}")

大分県大分市旦野原


### クローリング演習

#### 食べログ飲食店データ取得

##### 1. 取得したいレビューのURLの取得

In [105]:
baseurl = "https://tabelog.com"
URL=baseurl+"/oita/A4402/A440202/44005660/dtlrvwlst/"

In [112]:
URL

'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/'

In [106]:
res = requests.get(URL)

In [107]:
tree = html.fromstring(res.content)

In [108]:
eles = tree.xpath("//a[@class='rvw-item__title-target']")

In [110]:
reviewurls = []
for ele in eles:
    aurl = baseurl+ele.get("href")
    reviewurls.append(aurl)

In [261]:
set(reviewurls)

{'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B111965666/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B382464585/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B417580252/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B440585631/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B440868409/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B453057681/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B458471005/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B478849487/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B482850549/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B494269956/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B84684293/?use_type=0&smp=1'}

#### 2. 各レビューから取得したい内容のみを取得
- userid, name, rating, title, review
- 1つのURLでやってみる

In [155]:
rurl = reviewurls[0]
res = requests.get(rurl)
tree = html.fromstring(res.content)

In [156]:
#res.content

In [212]:
# User id/ Name
eles = tree.xpath("//a[@class='rvw-item__rvwr-name']")

In [213]:
eles[0].get("href")

'/rvwr/025510967/'

In [214]:
eles[0].getchildren()[0].text

'べっぴぃえんちゃん'

In [215]:
# rating
eles = tree.xpath("//div[@class='rvw-item__rvw-info']//b")

In [216]:
eles[0].text

'4.0'

In [217]:
#title
eles = tree.xpath("//p[@class='rvw-item__title']/strong")

In [218]:
eles[0].text

'いつも新たな出会いのあるお店'

In [242]:
# review
#eles = tree.xpath("//div[@class='rvw-item__rvw-comment']")
#eles = tree.xpath("//div[contains(@class, 'rvw-item') and contains(@class, 'rvw-comment')]")
eles = tree.xpath("//div[contains(@class, 'rvw-comment')]/p")

In [243]:
eles

[<Element p at 0xffff4708d3b0>, <Element p at 0xffff46fe2cb0>]

In [246]:
rawreview = eles[0].xpath("string()")

In [247]:
rawreview

'\n          3ヶ月ぶりの『トムヤムクン。』。お伺いするのは5、6回目だと思うが、来る度に新しい料理との出会いがある。メニューがなく、その時々の旬のもの、美味しいものを出してくれるシェフのおかげだ。アペリティフからして超独創的。こんな冷麺、見たことも食べたこともない。アジ、サバ、アサリの出汁に、新生姜とガスパチョ風のキュウリ。これだけでも斬新だが、ホタテの醤油漬けと一緒に食べると、あら不思議。塩ラーメンが一転醤油ラーメンに。本当に初めての味わいだ。次はクロダイのサラダ。コリンキーやズッキーニ、トマトはピクルスに。そして水菜、パクチーなど他の野菜やクロダイと一緒にキウイフルーツのソースで食べる。こんな発想聞いたことがない。魚は、一皿めはヘダイ。揚げたヘダイにとうもろこしのソース。とうもろこしが甘い。二皿めはイトヨリダイ。1番下に新タマネギのシャリアピンソース。魚の上には細かく刻んだナスとつるむらさき。オレガノも載っている。おそらくこれが味の決め手だろう。最後になって漸くタイ料理らしくなってきた（笑）まず看板料理のプーパッポンカリー。〆はエビのレッドカレー。レアなエビが美味しい。個人的にはもう少し辛くても良いが。まだまだ食べ足りない気もするが、ワインも空いたことだし、今日はここまで。ご馳走さまでした。今度は7皿でお願いするぞ〜\n        '

In [249]:
review = rawreview.strip()

In [250]:
# ここまでの内容を関数化
def scraper(rurl):
    res = requests.get(rurl)
    tree = html.fromstring(res.content)
    # User id/ Name
    eles = tree.xpath("//a[@class='rvw-item__rvwr-name']")
    uid = eles[0].get("href")
    name = eles[0].getchildren()[0].text
    # rating
    eles = tree.xpath("//div[@class='rvw-item__rvw-info']//b")
    rating = eles[0].text
    #title
    eles = tree.xpath("//p[@class='rvw-item__title']/strong")
    title = eles[0].text
    # review
    #eles = tree.xpath("//div[@class='rvw-item__rvw-comment']")
    #eles = tree.xpath("//div[contains(@class, 'rvw-item') and contains(@class, 'rvw-comment')]")
    eles = tree.xpath("//div[contains(@class, 'rvw-comment')]/p")
    rawreview = eles[0].xpath("string()")
    review = rawreview.strip()
    
    return uid, name, rating, title, review

In [262]:
rows = [["uid","name","rating", "title", "review"]]
for rurl in list(set(reviewurls)):
    row = scraper(rurl)
    rows.append(row)

In [367]:
import pandas as pd
pd.options.display.max_colwidth = 250

df = pd.DataFrame(rows[1:],columns=rows[0])

In [368]:
df

,name,text
0,いしのこ,大昔に買ったマスキングテープがミャクミャクカラーだったw\n\n#ミャクミャク #フィギュア #ガチャ
1,イトヲ,ミャクミャク様500円交換してきた
2,ぼのsky,万博のテストラン、色んなミャクミャクさまが見れてそれだけで大はしゃぎだった。かわいい。
3,にょろぴよ,これはトレーナーが仕事中に郵便局へ走って取り替えてもらったミャクミャク様記念硬貨だよ\n\nつついちゃダメよ
4,町田奈桜,ミャクミャク様硬貨を引き換えました。
...,...,...
95,サキツルギ𓅯,屋根裏部屋みたいなカフェで店のミャクミャク様と相席されている
96,えぃばる,3/20☀️\n能勢電鉄🚃滝山〜絹延橋🚉撮って出し(1/3)です😋\n能勢電鉄の運用が良かったので撮影に🤭\n1757Fの祝電に5146Fさくら🌸ヘッドマークに5148Fさくら🌸ヘッドマーク付きは何故か回送で🤔\nそして、ミャクミャク7200Fです🤗
97,もんもん@12・18大安,ミャクミャク様のフレーム可愛いwww
98,はなみ 茉莉@4月は創作スローペース,リリーちゃん、ミャクミャクさまだよ\nミャクミャクさま、リリーちゃんだよ


### BlueSkyデータの取得（API）

#### APIから投稿してみる

In [14]:
from atproto import Client
import pandas as pd

In [8]:
import json

with open("bsky_account.json", 'r') as f:
    acdic = json.load(f)

In [9]:
client = Client(base_url='https://bsky.social')
client.login(acdic["account"], acdic["password"])

ProfileViewDetailed(did='did:plc:7sqtnkepsjplcvuqpvprsx52', handle='bcoredsclass0001.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:7sqtnkepsjplcvuqpvprsx52/bafkreif4w23vqx2keibm2z665tm4i77kewnamclqtompktx6z6acdl2jdq@jpeg', banner=None, created_at='2025-04-10T00:59:16.772Z', description=None, display_name='', followers_count=0, follows_count=1, indexed_at='2025-04-10T00:59:16.772Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=1, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [10]:
client.send_post("世界の国から、こんにちは")

CreateRecordResponse(uri='at://did:plc:7sqtnkepsjplcvuqpvprsx52/app.bsky.feed.post/3lnuk6ozjqv2b', cid='bafyreidrml22pxd4npkpykzwzs5gmqiu2xdspd7spmaasnavowx7funbga')

#### キーワード検索

In [11]:
res = client.app.bsky.feed.search_posts({"q":"ミャクミャク", "sort":"top", "limit":100, "cursor":None})

In [12]:
print(res.cursor)
print(res.hits_total)

rows = [["name","text"]]
for post in res.posts:
#    print(post.cid)
#    print(post.author.did)
#    print(post.author.handle)
#    print(post.author.display_name)
#    print(post.author.avatar)
#    print(post.record.text)
#    print(post.like_count)
#    print(post.quote_count)
#    print(post.reply_count)
#    print(post.repost_count)
#    print(post.uri)
    name = post.author.display_name
    text = post.record.text.strip()
    row = [name, text]
    rows.append(row)

100
None


In [15]:
df = pd.DataFrame(rows[1:], columns=rows[0])

In [16]:
df

,name,text
0,Sheep🐑,つい先程酔っぱらいながら描いたミャクミャク擬人化♀\n#イラスト #大阪・関西万博 #ミャクミャク
1,鹿野ケル_Skeb募集中,ミャクミャクぽい子\n#illustration #イラスト
2,赤リス/Akarisu,ミャクミャクちゃーん
3,ガンギマリちゃん,ミャクミャク様、雑魚側なの…ｗ
4,青空ニュース🦋,【コラボ】『北斗の拳』原哲夫氏、万博ステージで「ケンシロウ×ミャクミャク」描く\nnews....
...,...,...
95,稔さんは日本語ﾁｮﾄﾜｶﾙ,むっちりしたミャクミャク様がほしい
96,繭良,情報番組見てたらミャクミャク様とサンリオのコラボぬいぐるみがチラっと映って良いな欲しなと思っ...
97,YMC🦪（ﾁﾙ）ゆる断酒day58,年長さんくらいの子どもつれた親が子どもと同じくらいのサイズのミャクミャクぬいを抱っこして電車...
98,青空ニュース🦋,【発表】大阪・関西万博、来場者が100万人突破 開幕11日目で達成\nnews.livedo...


### ES基盤から検索してみる（研究室のサーバに負荷がかかるので注意）

In [345]:
from elasticsearch import Elasticsearch
import datetime as dt
from dateutil.relativedelta import relativedelta

In [346]:
# Elasticsearch の接続先
es = Elasticsearch("http://cicero.csis.oita-u.ac.jp:9200")

In [355]:
def get_post_uris(query, start, end):
    # クエリ条件に合致する投稿を取得（例では commit.record.text に対して match クエリ）
    body = {
        "size": 1000,  # 必要件数に応じて調整
        "query": {
            "bool": {
                "must": [
                    {"match": {"commit.record.text": query}},
                    {"range": {"commit.record.createdAt": {"gte": start, "lte": end}}}
                ]
            }
        }
    }
    res = es.search(index="postindex-*", body=body)
    posts = []  # 各投稿の情報（URI やテキストなど）
    uris = []   # 再構築した URI のリスト
    for hit in res["hits"]["hits"]:
        src = hit["_source"]
        try:
            did  = src["did"]
            coll = src["commit"]["collection"]
            rkey = src["commit"]["rkey"]
            # URI を "at://{did}/{collection}/{rkey}" として再構築
            uri = f"at://{did}/{coll}/{rkey}"
            uris.append(uri)
            # ここではテキスト（全文の一部など）も取得しておく（任意）
            text = src["commit"]["record"].get("text", "")
            posts.append({"uri": uri, "text": text})
        except Exception as e:
            continue
    return posts, uris

In [356]:
def get_like_ranking(query, start, end):
    posts, uris = get_post_uris(query, start, end)
    if not uris:
        return pd.DataFrame()

    body = {
        "size": 0,
        "query": {
            "bool": {
                "filter": [
                    {"terms": {"commit.record.subject.uri": uris}},
                    {"term": {"commit.operation": "create"}},
                    {"range": {"commit.record.createdAt": {"gte": start, "lte": end}}}
                ]
            }
        },
        "aggs": {
            "post_likes": {
                "terms": {
                    "field": "commit.record.subject.uri",
                    "size": len(uris)
                }
            }
        }
    }
    res = es.search(index="likeindex-*", body=body)
    buckets = res["aggregations"]["post_likes"]["buckets"]
    df_aggs = pd.DataFrame([{"uri": b["key"], "like_count": b["doc_count"]} for b in buckets])
    posts_df = pd.DataFrame(posts)
#    st.dataframe(posts_df)
    merged = pd.merge(posts_df, df_aggs, on="uri", how="left").fillna(0)
    merged["like_count"] = merged["like_count"].astype(int)
    merged = merged.sort_values("like_count", ascending=False)
    return merged

In [357]:
today = dt.date.today()

In [358]:
start_date = today + relativedelta(days=-2)

In [359]:
start_date

datetime.date(2025, 4, 8)

In [360]:
start_date_str = start_date.strftime("%Y-%m-%dT00:00:00Z")

In [361]:
start_date_str

'2025-04-08T00:00:00Z'

In [369]:
end_date_str = today.strftime("%Y-%m-%dT23:59:59Z")

In [370]:
df = get_like_ranking(query="ミャクミャク", start=start_date_str, end=end_date_str)

In [371]:
df

,uri,text,like_count
108,at://did:plc:tr6khfvp7au7ohtjw23gqm3k/app.bsky.feed.post/3lmbwlzbuh22s,いまのところ万博に行く予定はないけど、ミャクミャクさまが好きすぎるので500円もらってきた🙌\n和暦音痴なので、今が令和7年ということを知れたのも大きな収穫だった😽\n私が末代だけど、末代までの宝にします🙏,37
101,at://did:plc:oz7hng4vov6msiu7ghfw3ywk/app.bsky.feed.post/3lmduu467xs2t,万博記念500円硬貨が発行された様ですが…\n\n「私の場合ミャクミャクじゃなくて春麗の500円硬貨出てくれればグッズとして収集するから結果としてめっちゃお金貯まりそう…」\n\nと思いました。,26
115,at://did:plc:3mkpgdviy6ewosxgyqxlrh5u/app.bsky.feed.post/3lmek5alwws2s,４月５日６日に大阪万博テストランに参加して来ました😆\n\n西ゲートから東ゲートまで歩いてだいたい30分ぐらい😆\n\n１日目は西ゲートから入り\n\n入り口でお迎えミャクミャクがかわいい😆❤️またすぐ近くにアトムがありガンダムがー😆そしてテレビで見ていた大屋根リングが感動😆立派でした😆❤️\n\n日本からはヘルスケアパビリオンの未来の洗濯機や睡眠など見学😆\n\n①必ず行かれる方はパビリオンを予約して回ってください♪\n\n②全てキャッシュレスです。\n\n③手荷物検査などがありかなり...,11
106,at://did:plc:k4xxxaazxkuvykifslorcvli/app.bsky.feed.post/3lmba3mefzc22,郵便局でミャクミャク500円もらってきたよぉ🥺🪙\n最悪並んでるかなと思ってたけど長閑なもんで誰もおらんかった🥺\n隣の信用金庫（取り扱い店舗一覧に無いマイナーなやつ）にはじいさんが何人かいたのだけど🥺,10
80,at://did:plc:bhv4gtgm2tarqoiaxsxl6zyi/app.bsky.feed.post/3lmbv764l322t,ミャクミャク様は万博を内部崩壊させて、なにかべつのちゃんとしたアンバサダーキャラクターとして転生して欲しい (´･ω･`),9
...,...,...,...
43,at://did:plc:urm5jkqllyhgwcfbhk6dxecu/app.bsky.feed.post/3lmbll6rts22f,ミャークミャクミャク！\nきさまらじんるいをすべてミャクミャクにしてやるミャク！,0
42,at://did:plc:pxe7c67kc66xmmfm3khge53j/app.bsky.feed.post/3lme5bn2itc2c,ミャクミャクの500円玉のことすっかり頭から抜けてた、、、,0
40,at://did:plc:y6a2heya72tueenawefij3fm/app.bsky.feed.post/3lmcrgob77c2l,ミャクミャク様の500円無事ゲットできた✨✨\nかわー😍,0
39,at://did:plc:ktkitaashhph3aaizrlshkdj/app.bsky.feed.post/3lmblrrhgb22w,x.com/suekichiii/s...\nミライトワ・ソメイティとミャクミャクの共演。なんかかわいい。,0
